In [8]:
import pymongo
import requests

client = pymongo.MongoClient("my URI")

db = client.sample_mflix
collection = db.movies

# Find the first 5 items
items = collection.find().limit(5)

for item in items:
    print(item)


{'_id': ObjectId('573a1390f29313caabcd516c'), 'plot': 'Original advertising for the film describes it as a drama of primitive life on the shores of the North Pacific...', 'genres': ['Drama', 'History'], 'runtime': 65, 'cast': ['Stanley Hunt', 'Sarah Constance Smith Hunt', 'Mrs. George Walkus', "Paddy 'Malid"], 'num_mflix_comments': 1, 'poster': 'https://m.media-amazon.com/images/M/MV5BMjE3MjAyNzM5NV5BMl5BanBnXkFtZTgwMjA5OTg5NjE@._V1_SY1000_SX677_AL_.jpg', 'title': 'In the Land of the Head Hunters', 'lastupdated': '2015-09-16 12:11:37.770000000', 'languages': ['English'], 'released': datetime.datetime(1914, 12, 7, 0, 0), 'directors': ['Edward S. Curtis'], 'writers': ['Edward S. Curtis (story)'], 'awards': {'wins': 1, 'nominations': 0, 'text': '1 win.'}, 'year': 1914, 'imdb': {'rating': 5.8, 'votes': 223, 'id': 4150}, 'countries': ['USA'], 'type': 'movie', 'tomatoes': {'viewer': {'rating': 2.7, 'numReviews': 64, 'meter': 18}, 'dvd': datetime.datetime(2000, 8, 15, 0, 0), 'website': 'http:

In [2]:
!pip3 install pymongo

     |████████████████████████████████| 534 kB 4.2 MB/s eta 0:00:01
  Using cached dnspython-2.6.1-py3-none-any.whl (307 kB)


In [9]:
hf_token = "....."
embedding_url = "https://api-inference.huggingface.co/pipeline/feature-extraction/sentence-transformers/all-MiniLM-L6-v2"

def generate_embedding(text: str) -> list[float]:

    response = requests.post(
        embedding_url,
        headers={"Authorization": f"Bearer {hf_token}"},
        json={"inputs": text})

    if response.status_code != 200:
        raise ValueError(f"Request failed with status code {response.status_code}: {response.text}")

    return response.json()

In [10]:
# generate the embedding vector
print(generate_embedding("freeCodeCamp is awesome"))

[-0.04656720161437988, -0.10088976472616196, 0.023876406252384186, 0.03345140069723129, 0.04757402837276459, 0.009101650677621365, -0.01100260391831398, -0.055370427668094635, -0.00289178011007607, 0.033230096101760864, 0.04593603312969208, 0.02030876651406288, -0.018762925639748573, 0.022110996767878532, 0.014108472503721714, 0.019433695822954178, -0.002060073195025325, 0.04771622642874718, 0.028062671422958374, -0.04205813631415367, -0.08776022493839264, -0.10249533504247665, -0.007292993366718292, 0.027418116107583046, 0.09413713961839676, 0.10750176012516022, -0.010373201221227646, 0.023484326899051666, 0.052364930510520935, -0.00033178081503137946, -0.02072683535516262, 0.012178347446024418, 0.041434723883867264, 0.0474986657500267, 0.01721927523612976, -0.006702423561364412, 0.015687284991145134, -0.03740110620856285, -0.14948050677776337, -0.05766904354095459, -0.12188048660755157, 0.10859912633895874, 0.05935731902718544, 0.03546006232500076, -0.013562660664319992, -0.015600200

In [11]:
for doc in collection.find({'plot':{"$exists": True}}).limit(50):
    doc['plot_embedding_hf'] = generate_embedding(doc['plot'])
    collection.replace_one({'_id': doc['_id']}, doc)

In [13]:
# try out and use the embeddings

query = "imaginary characters from outer space at war"

results = collection.aggregate([
  {"$vectorSearch": {
    "queryVector": generate_embedding(query),
    "path": "plot_embedding_hf",
    "numCandidates": 100,
    "limit": 4,
    "index": "default",
      }}
]);

for document in results:
    print(f'Movie Name: {document["title"]},\nMovie Plot: {document["plot"]}\n')

Movie Name: Westfront 1918,
Movie Plot: A group of German infantrymen of the First World War live out their lives in the trenches of France. They find brief entertainment and relief in a village behind the lines, but primarily ...

Movie Name: In the Land of the Head Hunters,
Movie Plot: Original advertising for the film describes it as a drama of primitive life on the shores of the North Pacific...

Movie Name: Tabu: A Story of the South Seas,
Movie Plot: In his final film, F.W. Murnau presents the tale of two young lovers on the idyllic island of Bora Bora in the South Pacific. Their life is shattered when the old warrior declares the girl ...

Movie Name: Wild Oranges,
Movie Plot: Millie Stope lives with her grandfather on a remote island. Her grandfather fled there for political reasons. But they're not alone. An escaped prisoner, Nicholas, is terrorizing them, and ...

